What This Function Does
This intelligent function automatically scans your entire Unity Catalog to identify and classify PII data for CCPA compliance. It provides a comprehensive data inventory that compliance teams need for regulatory reporting and consumer rights management.
🔍 Multi-Source PII Detection
The function uses a three-tiered detection approach (in priority order):

🏷️ Column Tags (Highest Priority)

Scans Unity Catalog column tags for PII classifications
Example: pii_type = 'email', data_class = 'phone'


📝 Column Comments/Descriptions (Medium Priority)

Analyzes column comments for PII keywords
Example: "Customer email address", "Social security number"


🔤 Column Names (Fallback)

Pattern matching on column names
Example: email_address, phone_number, customer_ssn

⚖️ Consumer Rights Analysis
Determines which CCPA consumer rights apply to each data element:

Right to Know - What personal information is collected
Right to Delete - Request deletion of personal information
Right to Opt-Out - Stop sale/sharing of personal information


Function signature

In [0]:
%sql
-- Set context to your specific catalog and schema
USE CATALOG sumitsaraswat_catalog;
USE SCHEMA compliance;

-- CCPA Data Inventory Function , uses column names, descriptions, AND tags
CREATE OR REPLACE FUNCTION sumitsaraswat_catalog.compliance.ccpa_data_inventory()
RETURNS TABLE(
  table_location STRING,
  ccpa_category STRING,
  personal_info_elements ARRAY<STRING>,
  sale_or_sharing_status STRING,
  consumer_right_to_know BOOLEAN,
  consumer_right_to_delete BOOLEAN,
  consumer_right_to_opt_out BOOLEAN,
  business_purpose STRING,
  third_party_recipients STRING,
  retention_period STRING,
  sensitive_personal_info BOOLEAN,
  privacy_policy_disclosure STRING,
  implementation_gap STRING,
  detection_method STRING
)
RETURN (
  WITH enhanced_pii_discovery AS (
    -- Enhanced PII discovery using column names, descriptions, AND tags
    SELECT 
      c.table_catalog,
      c.table_schema,
      c.table_name,
      c.column_name,
      c.data_type,
      COALESCE(c.comment, '') as column_description,
      
      -- Get column tags if they exist
      COALESCE(ct.tag_value, '') as column_tags,
      
      -- Enhanced PII detection using multiple sources
      CASE 
        -- First check explicit PII tags
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(EMAIL|E_MAIL).*' THEN 'email'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(PHONE|TELEPHONE|MOBILE).*' THEN 'phone'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(NAME|FIRST_NAME|LAST_NAME).*' THEN 'name'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(SSN|SOCIAL_SECURITY).*' THEN 'ssn'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(CREDIT_CARD|PAYMENT|CARD_NUMBER).*' THEN 'credit_card'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(ADDRESS|STREET|LOCATION).*' THEN 'address'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(DOB|BIRTH_DATE|DATE_OF_BIRTH).*' THEN 'dob'
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(IP_ADDRESS|IP_ADDR).*' THEN 'ip_address'
        
        -- Then check column descriptions/comments
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(EMAIL|E.MAIL|ELECTRONIC.MAIL).*' THEN 'email'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(PHONE|TELEPHONE|MOBILE|CELL).*' THEN 'phone'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(FIRST.NAME|LAST.NAME|FULL.NAME|CUSTOMER.NAME).*' THEN 'name'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(SSN|SOCIAL.SECURITY|TAX.ID).*' THEN 'ssn'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(CREDIT.CARD|PAYMENT|CARD.NUMBER).*' THEN 'credit_card'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(ADDRESS|STREET|POSTAL|MAILING).*' THEN 'address'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(BIRTH.DATE|DOB|DATE.OF.BIRTH).*' THEN 'dob'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(IP.ADDRESS|INTERNET.PROTOCOL).*' THEN 'ip_address'
        
        -- Finally fall back to column name patterns
        WHEN LOWER(c.column_name) RLIKE '.*(email|e_mail).*' THEN 'email'
        WHEN LOWER(c.column_name) RLIKE '.*(phone|tel|mobile).*' THEN 'phone'
        WHEN LOWER(c.column_name) RLIKE '.*(first.*name|last.*name|full.*name).*' THEN 'name'
        WHEN LOWER(c.column_name) RLIKE '.*(ssn|social.*security).*' THEN 'ssn'
        WHEN LOWER(c.column_name) RLIKE '.*(card.*number|cc.*|credit.*).*' THEN 'credit_card'
        WHEN LOWER(c.column_name) RLIKE '.*(address|street|city|zip).*' THEN 'address'
        WHEN LOWER(c.column_name) RLIKE '.*(birth.*date|dob|birthday).*' THEN 'dob'
        WHEN LOWER(c.column_name) RLIKE '.*(ip.*address|ip_addr).*' THEN 'ip_address'
        ELSE 'none'
      END as pii_type,
      
      -- Track how PII was detected for transparency
      CASE 
        WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(EMAIL|PHONE|NAME|SSN|CREDIT_CARD|ADDRESS|DOB|IP_ADDRESS).*' 
          THEN 'Detected via column tags'
        WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(EMAIL|PHONE|NAME|SSN|CREDIT|ADDRESS|BIRTH|IP).*' 
          THEN 'Detected via column description'
        WHEN LOWER(c.column_name) RLIKE '.*(email|phone|name|ssn|card|address|birth|ip).*' 
          THEN 'Detected via column name'
        ELSE 'No PII detected'
      END as detection_method
      
    FROM information_schema.columns c
    INNER JOIN information_schema.tables t 
      ON c.table_catalog = t.table_catalog 
      AND c.table_schema = t.table_schema 
      AND c.table_name = t.table_name
    LEFT JOIN system.information_schema.column_tags ct
      ON c.table_catalog = ct.catalog_name
      AND c.table_schema = ct.schema_name 
      AND c.table_name = ct.table_name
      AND c.column_name = ct.column_name
    WHERE c.table_catalog = 'sumitsaraswat_catalog'
      AND t.table_type IN ('MANAGED', 'EXTERNAL')
  ),
  
  pii_classified AS (
    SELECT 
      CONCAT(table_catalog, '.', table_schema, '.', table_name) as table_location,
      pii_type,
      COLLECT_LIST(column_name) as pii_columns,
      COLLECT_LIST(detection_method) as detection_methods,
      
      -- CCPA Category mapping
      CASE 
        WHEN pii_type IN ('email', 'phone', 'name', 'address', 'ssn') THEN 'Identifiers'
        WHEN pii_type IN ('dob') THEN 'Personal Information Categories'
        WHEN pii_type IN ('credit_card') THEN 'Commercial Information'
        WHEN pii_type IN ('ip_address') THEN 'Internet or Network Activity'
        ELSE 'Other Personal Information'
      END as ccpa_category,
      
      -- Sale/Sharing Status
      CASE 
        WHEN pii_type IN ('email', 'ip_address') THEN 'May be shared with marketing/analytics partners'
        WHEN pii_type IN ('phone', 'name', 'address') THEN 'Not sold, shared with service providers only'
        WHEN pii_type IN ('ssn', 'credit_card') THEN 'Never sold or shared (legal/service requirements only)'
        ELSE 'To be determined'
      END as sale_status,
      
      -- Consumer Rights
      TRUE as right_to_know,
      CASE 
        WHEN pii_type = 'ssn' THEN FALSE  -- Limited due to legal requirements
        ELSE TRUE 
      END as right_to_delete,
      CASE 
        WHEN pii_type IN ('email', 'ip_address', 'ssn') THEN TRUE
        ELSE FALSE
      END as right_to_opt_out,
      
      -- Business Purpose
      CASE 
        WHEN pii_type = 'email' THEN 'Customer communication, marketing, account management'
        WHEN pii_type = 'phone' THEN 'Customer service, account verification, security'
        WHEN pii_type = 'name' THEN 'Account management, service delivery, legal compliance'
        WHEN pii_type = 'ssn' THEN 'Legal compliance, tax reporting, identity verification'
        WHEN pii_type = 'credit_card' THEN 'Payment processing, fraud prevention'
        WHEN pii_type = 'address' THEN 'Service delivery, billing, shipping'
        WHEN pii_type = 'ip_address' THEN 'Security, analytics, fraud prevention, user experience'
        ELSE 'Business operations'
      END as business_purpose,
      
      -- Third Party Recipients
      CASE 
        WHEN pii_type = 'email' THEN 'Email service providers, Marketing platforms'
        WHEN pii_type = 'phone' THEN 'Customer service platforms, SMS providers'
        WHEN pii_type = 'name' THEN 'Service providers, Payment processors'
        WHEN pii_type = 'ssn' THEN 'Government agencies, Tax processors'
        WHEN pii_type = 'credit_card' THEN 'Payment processors, Fraud detection services'
        WHEN pii_type = 'address' THEN 'Shipping providers, Service technicians'
        WHEN pii_type = 'ip_address' THEN 'Analytics providers, Security services, CDN providers'
        ELSE 'Service providers'
      END as third_parties,
      
      -- Retention Period
      CASE 
        WHEN pii_type IN ('email', 'phone', 'address') THEN '24 months after last interaction'
        WHEN pii_type IN ('name', 'ssn') THEN '7 years for business records'
        WHEN pii_type = 'credit_card' THEN '90 days maximum (encrypted)'
        WHEN pii_type = 'ip_address' THEN '12 months'
        ELSE 'Under review'
      END as retention_period,
      
      -- Sensitive Personal Info Flag
      CASE 
        WHEN pii_type IN ('ssn', 'credit_card') THEN TRUE
        ELSE FALSE
      END as sensitive_flag,
      
      -- Privacy Policy Disclosure
      CASE 
        WHEN pii_type = 'email' THEN 'Email addresses collected for communication and marketing purposes'
        WHEN pii_type = 'phone' THEN 'Phone numbers used for customer service and security verification'
        WHEN pii_type = 'name' THEN 'Names used for account identification and service delivery'
        WHEN pii_type = 'ssn' THEN 'SSN collected only when legally required for tax/compliance'
        WHEN pii_type = 'credit_card' THEN 'Payment information used solely for transaction processing'
        WHEN pii_type = 'address' THEN 'Addresses used for service delivery and billing'
        WHEN pii_type = 'ip_address' THEN 'IP addresses collected for security and analytics purposes'
        ELSE 'Personal information used for business purposes'
      END as policy_disclosure,
      
      -- Implementation Gap Analysis
      CASE 
        WHEN pii_type IN ('email', 'ip_address') AND table_location LIKE '%marketing%' 
          THEN 'Opt-out mechanism required for marketing data'
        WHEN pii_type IN ('ssn', 'credit_card') 
          THEN 'Enhanced security controls verification needed'
        WHEN pii_type != 'none' AND table_location LIKE '%temp%' 
          THEN 'Retention policy definition required'
        WHEN pii_type != 'none' AND table_location LIKE '%backup%' 
          THEN 'Backup deletion process needed'
        ELSE 'Implementation appears complete'
      END as implementation_gap
      
    FROM enhanced_pii_discovery
    WHERE pii_type != 'none'
    GROUP BY 
      table_catalog, table_schema, table_name, pii_type
  )
  
  SELECT 
    table_location,
    ccpa_category,
    pii_columns as personal_info_elements,
    sale_status as sale_or_sharing_status,
    right_to_know as consumer_right_to_know,
    right_to_delete as consumer_right_to_delete,
    right_to_opt_out as consumer_right_to_opt_out,
    business_purpose,
    third_parties as third_party_recipients,
    retention_period,
    sensitive_flag as sensitive_personal_info,
    policy_disclosure as privacy_policy_disclosure,
    implementation_gap,
    ARRAY_JOIN(ARRAY_DISTINCT(detection_methods), ', ') as detection_method
  FROM pii_classified
  ORDER BY sensitive_flag DESC, ccpa_category, table_location
);

-- Example of how to tag columns for PII detection
-- You can run these commands to tag your existing columns:

/*
-- Tag columns with PII types
ALTER TABLE your_schema.your_table ALTER COLUMN user_id SET TAGS ('pii_type' = 'none');
ALTER TABLE your_schema.your_table ALTER COLUMN customer_email_addr SET TAGS ('pii_type' = 'email');
ALTER TABLE your_schema.your_table ALTER COLUMN contact_phone SET TAGS ('pii_type' = 'phone');
ALTER TABLE your_schema.your_table ALTER COLUMN customer_full_name SET TAGS ('pii_type' = 'name');
ALTER TABLE your_schema.your_table ALTER COLUMN billing_address_line1 SET TAGS ('pii_type' = 'address');
ALTER TABLE your_schema.your_table ALTER COLUMN social_security_num SET TAGS ('pii_type' = 'ssn');

-- Add column descriptions/comments that will be detected
ALTER TABLE your_schema.your_table ALTER COLUMN mystery_column COMMENT 'Contains customer email addresses for marketing';
ALTER TABLE your_schema.your_table ALTER COLUMN col123 COMMENT 'Customer phone number for support contact';
ALTER TABLE your_schema.your_table ALTER COLUMN data_field COMMENT 'Social security number for tax reporting';
*/

/*
-- Test query to see what the enhanced detection finds
CREATE OR REPLACE VIEW sumitsaraswat_catalog.compliance.pii_detection_test AS
SELECT 
  table_schema,
  table_name,
  column_name,
  COALESCE(comment, 'No description') as column_description,
  COALESCE(ct.tag_value, 'No tags') as column_tags,
  CASE 
    WHEN UPPER(COALESCE(ct.tag_value, '')) RLIKE '.*(EMAIL|PHONE|NAME|SSN|CREDIT_CARD|ADDRESS|DOB|IP_ADDRESS).*' 
      THEN 'PII detected via tags'
    WHEN UPPER(COALESCE(c.comment, '')) RLIKE '.*(EMAIL|PHONE|NAME|SSN|CREDIT|ADDRESS|BIRTH|IP).*' 
      THEN 'PII detected via description'
    WHEN LOWER(c.column_name) RLIKE '.*(email|phone|name|ssn|card|address|birth|ip).*' 
      THEN 'PII detected via column name'
    ELSE 'No PII detected'
  END as detection_result
FROM information_schema.columns c
LEFT JOIN system.information_schema.column_tags ct
  ON c.table_catalog = ct.catalog_name
  AND c.table_schema = ct.schema_name 
  AND c.table_name = ct.table_name
  AND c.column_name = ct.column_name
WHERE c.table_catalog = 'sumitsaraswat_catalog'
ORDER BY detection_result DESC, table_schema, table_name, column_name;
*/

Executing subquery: -- Set context to your specific catalog and schema
USE CATALOG sumitsaraswat_catalog.
Executing subquery: USE SCHEMA compliance.
Executing subquery: -- CCPA Data Inventory Function , uses column names, descriptions, AND tags
CREATE OR REPLACE FUNCTION sumitsaraswat_catalog.compliance.ccpa_data_inventory()
RETURNS TABLE(
  table_location STRING,
  ccpa_category STRING,
  personal_info_elements ARRAY<STRING>,
  sale_or_sharing_status STRING,
  consumer_right_to_know BOOLEAN,
  consumer_right_to_delete BOOLEAN,
  consumer_right_to_opt_out BOOLEAN,
  business_purpose STRING,
  third_party_recipients STRING,
  retention_period STRING,
  sensitive_personal_info BOOLEAN,
  privacy_policy_disclosure STRING,
  implementation_gap STRING,
  detection_method STRING
)
RETURN (
  WITH enhanced_pii_discovery AS (
    -- Enhanced PII discovery using column names, descriptions, AND tags
    SELECT 
      c.table_catalog,
      c.table_schema,
      c.table_name,
      c.column_name,

### Example usage

In [0]:
%sql
--Usage Examples
-- Example 1: Complete PII Inventory
USE CATALOG sumitsaraswat_catalog;
USE SCHEMA compliance;

-- Get comprehensive PII inventory across all tables
SELECT 
    table_location,
    ccpa_category,
    personal_info_elements,
    sensitive_personal_info,
    detection_method
FROM ccpa_data_inventory()
ORDER BY sensitive_personal_info DESC, ccpa_category;


table_location,ccpa_category,personal_info_elements,sensitive_personal_info,detection_method
sumitsaraswat_catalog.aep_billing.aep_usage_hist,Commercial Information,List(Bill Due Date),true,No PII detected
sumitsaraswat_catalog.sample_data.financial_transactions,Commercial Information,List(credit_card_number),true,Detected via column name
sumitsaraswat_catalog.sample_data.employee_records,Identifiers,List(social_security_number),true,No PII detected
sumitsaraswat_catalog.sample_data.marketing_campaigns,Identifiers,List(ip_address),false,Detected via column name
sumitsaraswat_catalog.sample_data.financial_transactions,Identifiers,List(customer_email),false,Detected via column name
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,List(phone_number),false,Detected via column name
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,"List(first_name, last_name)",false,Detected via column tags
sumitsaraswat_catalog.sample_data.marketing_campaigns,Identifiers,List(customer_email),false,Detected via column name
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,"List(street_address, city, zip_code)",false,"Detected via column name, No PII detected"
sumitsaraswat_catalog.sample_data.customer_profiles,Identifiers,List(email_address),false,Detected via column tags


In [0]:
%sql
--Example 2: High-Risk Data Focus
-- Focus on sensitive PII requiring enhanced security
SELECT 
    table_location,
    personal_info_elements,
    business_purpose,
    implementation_gap
FROM ccpa_data_inventory()
WHERE sensitive_personal_info = TRUE
ORDER BY table_location;


table_location,personal_info_elements,business_purpose,implementation_gap
sumitsaraswat_catalog.aep_billing.aep_usage_hist,List(Bill Due Date),"Payment processing, fraud prevention",Enhanced security controls verification needed
sumitsaraswat_catalog.sample_data.employee_records,List(social_security_number),"Legal compliance, tax reporting, identity verification",Enhanced security controls verification needed
sumitsaraswat_catalog.sample_data.financial_transactions,List(credit_card_number),"Payment processing, fraud prevention",Enhanced security controls verification needed
